In [48]:
# 1. Create a dataset that only contains passenger_count (4th col), pulocationid (8th col), dolocationid (9th col), and total_amount (17th col) based on the 2019-01-h1.csv dataset. In the Jupyter notebook, show the first 10 entries in the created dataset.

filePath = "gs://dataproc-staging-us-central1-176366540446-vnmuytv8/data/a4data.csv"
a4DF = spark.read.csv(filePath, header=True, inferSchema=True)
a4DF.show(10)

+---------------+------------+------------+------------+
|passenger_count|pulocationid|dolocationid|total_amount|
+---------------+------------+------------+------------+
|            1.0|       151.0|       239.0|        9.95|
|            1.0|       239.0|       246.0|        16.3|
|            3.0|       236.0|       236.0|         5.8|
|            5.0|       193.0|       193.0|        7.55|
|            5.0|       193.0|       193.0|       55.55|
|            5.0|       193.0|       193.0|       13.31|
|            5.0|       193.0|       193.0|       55.55|
|            1.0|       163.0|       229.0|        9.05|
|            1.0|       229.0|         7.0|        18.5|
|            2.0|       141.0|       234.0|        13.0|
+---------------+------------+------------+------------+
only showing top 10 rows



In [49]:
# 2. Create trainDF and testDF.
trainDF, testDF = a4DF.randomSplit([.8, .2], seed=42)
print(f"There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set")

There are 2920360 rows in the training set, and 730639 in the test set


In [50]:
# 3. Create a decision tree regressor to predict total_amount from the other three features.
from pyspark.ml.regression import DecisionTreeRegressor

dtr = DecisionTreeRegressor(featuresCol="features", labelCol="total_amount").setMaxBins(300)

In [51]:
# 4. Create a pipeline.
from pyspark.ml import Pipeline

feature_cols = ["passenger_count","pulocationid","dolocationid"]
vecAssembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

pipeline = Pipeline(stages=[vecAssembler, dtr])

In [52]:
# 5. Train the model
pipelineModel = pipeline.fit(trainDF)

In [53]:
# 6. Show the predicted results along with the three features in the notebook.
from pyspark.sql.functions import rand 
# Rand() to show predictions without being sorted by features
predDF = pipelineModel.transform(testDF)
predDF.orderBy(rand()).select("features", "total_amount", "prediction").show(10)

+-----------------+------------+-----------------+
|         features|total_amount|       prediction|
+-----------------+------------+-----------------+
|[1.0,116.0,151.0]|         9.8|13.58510172986951|
| [1.0,43.0,238.0]|         9.8|13.58510172986951|
|[1.0,140.0,233.0]|         9.3|12.02480847692699|
|  [2.0,87.0,45.0]|         7.3|13.58510172986951|
| [1.0,43.0,230.0]|       16.56|13.58510172986951|
|  [2.0,50.0,48.0]|         8.5|13.58510172986951|
|  [1.0,79.0,90.0]|        11.0|13.58510172986951|
|[3.0,237.0,162.0]|         7.8|12.02480847692699|
|[6.0,143.0,163.0]|        7.56|12.02480847692699|
|[4.0,141.0,141.0]|         5.3|12.02480847692699|
+-----------------+------------+-----------------+
only showing top 10 rows



In [54]:
# 7. Evaluate the model with RMSE.
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol="total_amount",
    metricName="rmse"
)
rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE: {rmse}")

RMSE: 60.925502421048385
